In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [3]:
#Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon-books").getOrCreate()


In [4]:
#Reading in datafrom datasets stored from the cloud
from pyspark import SparkFiles 
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url) # adding the file into our spark context 
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True) #reading in the tsv file 

#show DataFrame
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
#grabbing the size of our data
books_df.count()

3105520

In [6]:
#cleaning up the data frame to match our schema 
from pyspark.sql.functions import to_date
review_id_df = books_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
|R2UP6XSVYJBJ2H|   24209213|0976822105|     731931430| 2005-10-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-10

In [8]:
#product table schema 
product_id_df = books_df.select(["product_id","product_title"])

product_id_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
|0316769487|The Catcher in th...|
|0805076069|Bait and Switch: ...|
|1581603681|Opening Combinati...|
|0300108834|A Little History ...|
|0446531081|Hour Game (King &...|
|0976822105|     Faith is a Verb|
|1580085695|Furry Logic: A Gu...|
|0029148510|Acts of War: Beha...|
|1592285570|Temple to the Win...|
|0811848833|The Star Wars Pos...|
|0782144276|Photoshop for Nat...|
|0375757996|The Basic Works o...|
|013146308X|The Pterosaurs: F...|
|0375701907|Straight Man: A N...|
|141378240X|   Lipstick's Legacy|
+----------+--------------------+
only showing top 20 rows



In [11]:
#customer table schema 
#grouping by the customer id, aggregating the count and renaming the column for the customer count 
customer_id_df = books_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")

customer_id_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52496855|             8|
|   12079026|             1|
|   49471923|             2|
|   40347396|            75|
|   12083267|             1|
|   25824372|             1|
|   29853725|             1|
|   12934400|             1|
|   51371372|             1|
|   37236260|            72|
|   50649342|             2|
|   49379318|             3|
|   39614932|             2|
|   12462751|             1|
|   50018404|             1|
|   50860080|            27|
|   38825828|            11|
|   45193257|           671|
|   49161738|             1|
|   43470228|             5|
+-----------+--------------+
only showing top 20 rows



In [12]:
#vine table schema 
vine_df = books_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            